seurat5 gene expression analysis

This R-suerat5 notebook analysis four sc-RNAseq samples. 
This notebook uses parts of: seurat5 pbmc vignette: https://satijalab.org/seurat/articles/pbmc3k_tutorial 
azimuth vignette: https://satijalab.github.io/azimuth/articles/run_azimuth_tutorial.html


In [ ]:
#load all the required R packages previously installed in the conda env
library(dplyr)
library(Seurat)
library(tidyverse)
library(openxlsx)
library(ggplot2)
library(sctransform)
library(pheatmap)
library(ggrepel)
library(reticulate)
library(EnhancedVolcano)
library(clustree)
library(phateR)
library(Rmagic)
library(EnhancedVolcano)
library(stringr)
library(glmGamPoi)
library(presto)
library(SeuratData)
library(Azimuth)
library(patchwork)
sessionInfo()

getwd()

In [ ]:
# Generate directory name with current date
current_date <- format(Sys.Date(), "%Y%m%d")
outDir <- paste0(current_date, "_scRNAseq_results")
ifelse(dir.exists(outDir), NA, dir.create(outDir))



In [ ]:
# importing the cellranger count matrices from matrix folder
dir.vec <- list.dirs(path = "./matrix/")
dir.vec <- dir.vec[2:length(dir.vec)]
dir.vec
length(dir.vec)

In [ ]:
#assigning names to the folders based on the sample names provided by the customer
#Do not use underlines(_) in the sample names because seurat uses _ character to seperate sample name and index sequence
sampleList <- c('Fetalheart1','Fetalheart2')
names(dir.vec) <- sampleList
dir.vec

In [ ]:
#run read10x
myRawCount <- Read10X(dir.vec)

In [ ]:
#create initial seurat object using CreateSeuratObject
seurat_obj <- CreateSeuratObject(counts = myRawCount, project = "fetal_heart", min.cells = 3, min.features = 200)
head(seurat_obj@meta.data)

In [ ]:
#use MT/RP (upper case) for human and mt/rp (lower case) for mouse sequencing
seurat_obj <- PercentageFeatureSet(seurat_obj, pattern = "^MT-", col.name = "percent.mt")
seurat_obj <- PercentageFeatureSet(seurat_obj, pattern = "^RP[Sl][[:digit:]]", col.name = "percent.ribo")

In [ ]:
options(repr.plot.width=12, repr.plot.height=18)
#pdf("QC_VlnPlot.pdf",  height = 14, width = 18)
pdf(paste(outDir, "QC_VlnPlot.pdf", sep="/"),  height = 14, width = 18)
VlnPlot<-VlnPlot(seurat_obj, features = c("nFeature_RNA", "percent.ribo", "percent.mt"), ncol = 3)
VlnPlot
dev.off()
VlnPlot

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used
# for anything calculated by the object, i.e. columns in object metadata, PC scores etc.
pdf(paste(outDir, "QC_ScatterPlot.pdf", sep="/"),  height = 14, width = 18)
plot1 <- FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2
dev.off()
plot1
plot2

In [ ]:
#filtering nFeature_RNA and percent.mt on the basis of the above plots. for percent.mt where we see the "tailing" start
#for nFeature_RNA it is generally where the curve starts to staturate
seurat_obj <- subset(seurat_obj, subset = nFeature_RNA > 200 & nFeature_RNA < 10000 
                 & percent.mt < 8)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used
# for anything calculated by the object, i.e. columns in object metadata, PC scores etc.
pdf(paste(outDir, "QC_ScatterPlot_afterFiltering.pdf", sep="/"),  height = 14, width = 18)
plot1 <- FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2
dev.off()
plot1
plot2

In [ ]:
#normalizing using SCTransform
vars_to_regress <- c("nCount_RNA", "percent.mt")
system.time(suppressWarnings(seurat_obj <- SCTransform(seurat_obj,
                            method = "glmGamPoi",
                            vst.flavor = "v2",
                            vars.to.regress = vars_to_regress, 
                            verbose = FALSE)))

In [ ]:
#running first PCA to estimate the number of dimentions
seurat_obj <- RunPCA(seurat_obj, verbose = FALSE)
elbow_plot <- ElbowPlot(seurat_obj, ndims = 30)
elbow_plot

In [ ]:
#the ndims is estimated where the elobow is in the elbow plot. We use 20 dims based on the elbow plot
#the resolution is where the the clusters start to stabilize in the tree
seurat_obj <- RunPCA(seurat_obj, verbose = FALSE)
seurat_obj <- RunUMAP(seurat_obj, dims = 1:30, verbose = FALSE)
seurat_obj <- FindNeighbors(seurat_obj, dims = 1:30, verbose = FALSE)
seurat_obj <- FindClusters(seurat_obj, verbose = FALSE)

In [ ]:
#running FindClusters over a range of resolutions to determine a reasonable resolution to use
resolution.range <- seq(from = 0.2, to = 1.4, by = 0.2)
seurat_obj <- FindClusters(seurat_obj, verbose = FALSE, resolution = resolution.range)
clustree(seurat_obj, prefix = "SCT_snn_res.", node_colour = "sc3_stability")

In [ ]:
#looking at the tree.. the clusters seem to stablize at resolution 1-1.2. so we run FindClusters again at resolution 1
seurat_obj <- FindClusters(seurat_obj, verbose = FALSE, resolution = 1)
#saving the seurat object to reuse later
save(seurat_obj, file="seurat_obj_UMAP.Rdata")

In [ ]:
markers <- FindAllMarkers(object = seurat_obj, only.pos = TRUE,
                                    min.pct = 0.25,  thresh.use = 0.25, 
                                    assay = "SCT")

write.xlsx(markers, file=paste(outDir, "cluster_markers_SCT.xlsx", sep="/"), 
           overwrite = T, rowNames=T, colNames=T)

In [ ]:
#azimuth annotation
#download human heart reference from https://azimuth.hubmapconsortium.org/references/#Human%20-%20Heart
#extract the zip file and save the two index files in a heart/ folder
seurat_obj <- RunUMAP(seurat_obj, dims = 1:30, reduction = 'pca', reduction.name = 'AzimuthUMAP')
seurat_obj <- RunAzimuth(seurat_obj, reference = "heart/")
#looking at the annotation col names
head(seurat_obj@metadata)

In [ ]:
#running dimplots at Level1 and Level2 annotations and at cluster and identity levels
Index <- DimPlot(seurat_obj, reduction = 'AzimuthUMAP', group.by = 'orig.ident') + NoLegend() +  theme_minimal()
Cluster <- DimPlot(seurat_obj, reduction = 'AzimuthUMAP') + NoLegend() +  theme_minimal()
Level1 <- DimPlot(seurat_obj, reduction = 'AzimuthUMAP', group.by = 'predicted.celltype.l1', label = TRUE, repel = T) + NoLegend() +  theme_minimal()
Level2 <- DimPlot(seurat_obj, reduction = 'AzimuthUMAP', group.by = 'predicted.celltype.l2', label = TRUE, repel = T) + NoLegend() +  theme_minimal()


In [ ]:
#saving the UMAP plots
pdf(file = paste(outDir, "UMAP_bySample.pdf", sep="/"))
Index
dev.off()

pdf(file = paste(outDir, "UMAP_byCluster.pdf", sep="/"))
Cluster
dev.off()

pdf(file = paste(outDir, "UMAP_by_cellType_Level1.pdf", sep="/"))
Level1
dev.off()

pdf(file = paste(outDir, "UMAP_by_cellType_Level2.pdf", sep="/"))
Level2
dev.off()

In [ ]:
# here we are creating a function to find the most frequent Level2 marker and combing it with 

#Top 5 markers per cluster
top_markers <- markers %>%
  group_by(cluster) %>%
  top_n(n = 5, wt = avg_log2FC)

Get most frequent Azimuth annotation per cluster
get_mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

# Replace 'predicted.celltype' with the actual column name if different
cluster_annotations <- seurat_obj@meta.data %>%
  group_by(seurat_clusters) %>%
  summarize(predicted.celltype.l2 = get_mode(predicted.celltype.l2))

#Merge with top markers
summary_table <- left_join(top_markers, cluster_annotations, by = c("cluster" = "seurat_clusters"))

# Step 5: Save to table
write.xlsx(summary_table, file=paste(outDir, "azimuth_cluster_markers_L2.xlsx", sep="/"), 
           overwrite = T, rowNames=F, colNames=T)
